In [14]:
import pandas as pd

In [15]:
# Load the data
df = pd.read_csv('avgfp.tsv', sep='\t')

# Define the wildtype sequence for mutation translation
wildtype = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"

# Function to apply mutations to the wildtype sequence
def apply_mutations(wildtype, mutations):
    # Convert the wildtype string into a list for mutability
    sequence_list = list(wildtype)
    for mutation in mutations.split(','):
        pos = int(mutation[1:-1]) + 1  # Convert to 0-based index
        mut_residue = mutation[-1]
        sequence_list[pos] = mut_residue  # Apply the mutation
    return ''.join(sequence_list)

# Translate mutations from the 'variant' to 'sequence'
df['sequence'] = df['variant'].apply(lambda x: apply_mutations(wildtype, x))

# Rename columns
df.rename(columns={'score': 'functional_score'}, inplace=True)

# Select and reorder the relevant columns
df = df[['sequence', 'variant', 'functional_score']]

# Rename the 'variant' column to 'mutations' for final format
df.rename(columns={'variant': 'mutations'}, inplace=True)

# Save the processed DataFrame to a pickle file
df.to_pickle('SeqFxnDataset.pkl')

In [16]:
df['sequence'].iloc[0]

'MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRDEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK'

In [17]:
df['mutations'].iloc[0]

'A108D'

In [18]:
wildtype[109]

'A'

In [19]:
MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK
MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRDEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK



NameError: name 'MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHY' is not defined